In [ ]:
import numpy as np

from PIL import Image, ImageOps

import matplotlib.pyplot as plt

import h5py

from datetime import datetime

In [ ]:
import tensorflow as tf

tf.config.get_visible_devices('GPU')    

In [ ]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
#@title uploader
DIR = "18v2b6y59OqMo56enQa-j5oXLvoESG5jm" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=DIR)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': DIR }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

In [ ]:
P = []*0

DIR = "/content/data.h5"

files = h5py.File(DIR, 'r')

predcat, predsubcat = "c2", "031_002"

#predimg = np.array([1])

predimg = np.array(files["c2"]["031_002"][100:201, 100:200, :-1].flatten())

labels = []*0

x_rgb, y_label = []*0, []*0

for category in files.keys():

    for subcategory in files[category].keys():         

        if subcategory[:3] not in labels:

            labels.append(subcategory[:3])

        #dmpimg = np.array([1])

        dmpimg = np.array(files[category][subcategory][100:201, 100:200, :-1].flatten())
        
        x_rgb.append(dmpimg)
        
        y_label.append(files[category][subcategory][100:201, 100:200, -1][-1][-1])

x_rgb = np.array(x_rgb, dtype = np.uint8)

x_rgb2 = x_rgb

y_label = np.array(y_label)

from sklearn.utils import shuffle

x_rgb2 = shuffle(x_rgb2, random_state=0)

x_test = x_rgb2[:50]

y_test = []*0

for i in x_test:

    for j in range(x_rgb.shape[0]):

        if np.array_equal(i, x_rgb[j]) == True:

            y_test.append(y_label[j])

In [ ]:
regressor = LinearRegression()  
regressor.fit(x_rgb, y_label) 

In [ ]:
#To retrieve the intercept:
print(regressor.intercept_)

#For retrieving the slope:
print(regressor.coef_)

In [ ]:
y_pred = regressor.predict(x_rgb)
y_pred = np.round(y_pred)

print(np.array(list(zip(y_label, y_pred))))

In [ ]:
y_pred = regressor.predict(x_test)
y_pred = np.round(y_pred)
print(np.array(list(zip(y_test, y_pred))))